# Notebook similarity job demo

This shows how you might replace the Rails `notebook_summaries` job with Python using this library.

Note that we use our ORM interface to retrieve the notebooks, but then use SQLAlchemy's core interface to update the `notebook_summaries` table in the database.  SQLAlchemy ORM does support bulk operations, but using that interface is [discouraged](https://docs.sqlalchemy.org/en/14/orm/persistence_techniques.html#bulk-operations) in favor of the core interface -- plus we don't have any need to manipulate NotebookSummary models as objects anyway.

In [ ]:
import datetime

import pandas as pd
import sklearn
import sqlalchemy as sa
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nbgallery.database as nbgdb
import nbgallery.database.orm as nbgorm
import nbgallery.notebooks as nbgnb

## TF-IDF computation

We'll use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) from scikit-learn to generate a notebook-term matrix.

In [ ]:
session = nbgorm.Session()

In [ ]:
corpus = []
index = {} # matrix row/column id => notebook id
for i, nb in enumerate(session.query(nbgorm.Notebook).all()):
    index[i] = nb.id
    doc = nbgnb.from_model(nb)
    corpus.append(' '.join(doc.sources()))
index

In [ ]:
# We're including code, so to avoid getting numeric contants, let's change
# the default token pattern to require words start with a letter.
vectorizer = TfidfVectorizer(token_pattern=r'(?u)\b[a-z]\w+\b')
tfidf = vectorizer.fit_transform(corpus)
tfidf

## Job parameters

For each notebook, we'll keep the 5 most similar notebooks as long as the similarity score is > 0.1.

In [ ]:
keep_top_n = 5
min_score = 0.1

## Big memory version

We'll use scikit-learn's [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) to compute a notebook-notebook similarity matrix from the TF-IDF notebook-term matrix.

In this "big memory" version, we get a dense matrix back and then add all the database entries in one big bulk insert.

In [ ]:
doc_similarity = cosine_similarity(tfidf)
doc_similarity.shape

We'll convert to a dataframe just for some variety; we could leave it as a matrix and access directly.

In [ ]:
df = pd.DataFrame(doc_similarity)
df

Loop through and find the top n most similar notebooks for each notebook.

In [ ]:
now = datetime.datetime.now()
entries = [] # rows to be added to the database

for i, column in df.items():
    nbid_i = index[i]
    #print(f"column {i}, notebook {nbid_i}")
    top_n = []
    for j, score in column.sort_values(ascending=False, inplace=False).items():
        if len(top_n) == keep_top_n or score < min_score:
            break
        if i == j:
            continue
        nbid_j = index[j]
        top_n.append((nbid_j, score))
        entries.append({
            'notebook_id': nbid_i,
            'other_notebook_id': nbid_j,
            'score': score,
            'created_at': now, 
            'updated_at': now
        })
    #print(' ', top_n)

print(len(entries))
print(entries[0])

Clear out all the old entries in the table.

In [ ]:
table = nbgorm.NotebookSimilarity.__table__
nbgdb.engine.execute(table.delete())

Insert all the new entries.

In [ ]:
insert = table.insert().values(entries)
#print(insert.compile(dialect=sa.dialects.mysql.dialect()))
nbgdb.engine.execute(insert)

See what it looks like.

In [ ]:
pd.read_sql(table.select(), nbgdb.engine)

## Small memory version

In this "small memory" version, we'll ask for the cosine similarity as a sparse matrix.  Then we'll delete and insert once per notebook, instead of one giant delete and insert for the whole corpus.

In [ ]:
sparse_similarity = cosine_similarity(tfidf, dense_output=False)
sparse_similarity

In [ ]:
table = nbgorm.NotebookSimilarity.__table__

Again, lop through and find the top n most similar notebooks for each notebook, but update the database notebook by notebook.

In [ ]:
for i, row in enumerate(sparse_similarity):
    nbid_i = index[i]
    #print(f"column {i}, notebook {nbid_i}")
    now = datetime.datetime.now()
    entries = []
    top_n = []
    coo = row.tocoo()
    for j, score in sorted(zip(coo.col, coo.data), key=lambda z : z[1], reverse=True):
        if len(top_n) == keep_top_n or score < min_score:
            break
        if i == j:
            continue
        nbid_j = index[j]
        top_n.append((nbid_j, score))
        entries.append({
            'notebook_id': nbid_i,
            'other_notebook_id': nbid_j,
            'score': score,
            'created_at': now, 
            'updated_at': now
        })
    #print(' ', top_n)
    delete = table.delete().where(table.c.notebook_id == nbid_i)
    nbgdb.engine.execute(delete)
    insert = table.insert().values(entries)
    nbgdb.engine.execute(insert)

See what it looks like.

In [ ]:
pd.read_sql(table.select(), nbgdb.engine)